In [1]:
import os
import numpy as np
from tqdm import tqdm
import h5py
import torch
from torcheval.metrics import BinaryAccuracy,AUC

In [2]:
from collections import OrderedDict

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
from src.evaluate import Precision,Recall,F1_score
from src.model import build_model
from src.get_config import get_config

In [5]:
def normalization(rgb_img,mean_list=[0.485, 0.456, 0.406],std_list=[0.229, 0.224, 0.225]):
    MEAN = 255 * np.array(mean_list)
    STD = 255 * np.array(std_list)
    rgb_img = rgb_img.transpose(-1, 0, 1)
    norm_img = (rgb_img - MEAN[:, None, None]) / STD[:, None, None]
    
    return norm_img

In [6]:
def eval(outs,gts):
    
    precision_m = Precision()
    recall_m = Recall()
    bAcc_m = BinaryAccuracy()
    auc_m = AUC()

    f1_score = F1_score(outs.to(torch.float32),gts.to(torch.float32))
    precision = precision_m(outs.to(torch.float32),gts.to(torch.float32))
    recall = recall_m(outs.to(torch.float32),gts.to(torch.float32))

    acc=[]
    auc=[]
    
    for i in range(len(outs)):
        out = outs[i]
        gt = gts[i]
        
        # Binary Accuracy
        bAcc_m.update(out, gt)
        ba = bAcc_m.compute()
        acc.append(ba.item())
        
        # AUC
        auc_m.update(out, gt)
        auc_val = auc_m.compute()
        auc.append(auc_val.item())
        
    acc = np.array(acc)
    acc = np.mean(acc)
    
    auc = np.array(auc)
    auc = np.mean(auc)
    
    return acc, f1_score.item(), precision.item(), recall.item(), auc   

In [7]:
# load config
cfg_path = "configs/efficientNet_B0_celebA.py"
cfg = get_config(cfg_path)

In [8]:
# load model
model_path = "./save_model_224/best_epoch28.pth"

model = build_model(cfg.network,cfg.num_classes,'',False)
load_weight = torch.load(model_path)
new_state_dict = OrderedDict()
for n, v in load_weight.items():
    name = n.replace("module.","") 
    new_state_dict[name] = v
        
model.load_state_dict(new_state_dict)
_ = model.eval()

In [9]:
# dataset
hf_path = "celebA/h5/230720/celeba_224_test.h5"

hf = h5py.File(hf_path,'r')

label_list = [lb.decode('utf-8') for lb in hf['label_list']]
print(label_list)

images = hf['images']
labels = hf['labels']

['Beard', 'Smiling', 'Eyeglasses', 'Wearing_Lipstick', 'Wearing_Hat', 'Wearing_Earrings', 'Wearing_Necklace', 'Wearing_Necktie']


In [ ]:
outs=[]
gts=[]

for i in tqdm(range(len(images))):
    gts.append(labels[i])
    input_img = normalization(images[i])
    input_img = torch.tensor(np.expand_dims(input_img,0).astype(np.float32))
    out = model(input_img)
    out = out[0].detach().numpy()
    outs.append(out)

outs = torch.tensor(outs)
gts = torch.tensor(gts)

In [41]:
gts.shape

torch.Size([16568, 8])

In [43]:
acc, f1_score, precision, recall, auc = eval(outs,gts)

In [46]:
print("Acc:",round(acc*100,3),"%")
print("F1-score:",f1_score)
print("Precision:",precision)
print("Recall:",recall)
print("AUC:",auc)

Acc: 61.171 %
F1-score: 0.34590545296669006
Precision: 0.27919095754623413
Recall: 0.45451486110687256
AUC: 0.024653050414444398
